# Demonstration View

View a demonstration by sliding through the frames.
This also plots the z height.

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

# default parameter values
segment_height = False
segment_labels = False  # this seems deprecated
segment_imgheight = False
movement_threshold = None
keep_frames_method = "sparse"
gripper_close_steps = 30
demonstration_type = "grasp_insert"

#recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/lego", 3
#keep_frames_method = "dense"
#gripper_close_steps = 10
#segment_colors = [(1,0,0), (0,0,1), (0,0,1)]
#segment_thresholds = [.75,.60, .60]

# recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/wheel", 17
# segment_colors = ["bw", (1, 0, 0), (1, 0, 0)]
# segment_thresholds = [.30, .75, .75]
# segment_labels = (2, False, False)

#recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/pick_stow", 2
#segment_colors = [(0, 0, 1), "keep_black", "keep_black"]
#segment_thresholds = [.56, .45, .45]
#segment_imgheight = (False, 260, 260)

# recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/transfer_blue_block", 0
# demonstration_type = "grasp"
# segment_colors = [(0, 0, 1),(0, 0, 1)]
# segment_thresholds = [.56, .56]

# recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/transfer_orange", 0
# demonstration_type = "grasp"
# segment_colors = [(130/255, 80/255, 59/255),(130/255, 80/255, 59/255)]
# segment_thresholds = [.43, .43]

#recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/navigate_blue_letter_block", 0
#demonstration_type = "navigate"
#segment_colors = [(0, 0, 1),(0, 0, 1)]
#segment_thresholds = [.56, .56]

# recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/sick_vacuum/", 4
# keep_frames_skip = [(110,130)]
# segment_colors = [(0, 0, 1), "bw"]
# segment_thresholds = [.65, .70]

#recording, episode_num = "/home/argusm/sick_combine/", 3
#segment_colors = [(0, 0, 1), "keep_black"]
#segment_thresholds = [.60, .48]
#segment_imgheight = (False, 190)

recording, episode_num = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/car_screwdriver/", 0
demonstration_type = "grasp"
segment_colors = ["keep_black", "keep_black", "bw"]
segment_thresholds = [.67, .67, .59]

state_recording_fn = "{}/episode_{}.npz".format(recording, episode_num)
flow_recording_fn = "{}/episode_{}.npz".format(recording, episode_num)
state_recording = np.load(state_recording_fn)["robot_state_full"]
actions = np.load(state_recording_fn)["actions"]
ee_positions = state_recording[:,:3]
flow_recording = np.load(flow_recording_fn)["rgb_unscaled"]
try:
    seg_masks = np.load(flow_recording_fn)["seg_masks"]
except (KeyError,ValueError):
    seg_masks = None

print("loaded.")

# Keep Frames

decide which frames to keep, saved as per-frame boolean array.

1. Decisison based on gripper state
2. Decision based on (optinal subsampling)
3. Decision based on skip variable

In [ ]:
GRIPPER_OPEN = 1.0
GRIPPER_CLOSE = -1.0
DEMO_STEP = 20

# use actions here instead of state position recordings as these
# are more direct/ reliable
gr_actions = actions[:, -1]
num_frames = flow_recording.shape[0]-1
keysteps = np.where(np.diff(gr_actions))[0]

if demonstration_type == "navigate":
    gr_actions[-1] = GRIPPER_CLOSE
    demonstration_type = "grasp"

# keystep is the step just before closing
assert(gr_actions[keysteps[0]] == GRIPPER_OPEN)
assert(gr_actions[keysteps[0]+1] == GRIPPER_CLOSE)

# divide sequence into steps, defined by gripper action
segment_steps = np.zeros(gr_actions.shape)
segment_steps[keysteps+1] = 1
segment_steps = np.cumsum(segment_steps).astype(int)

# keystep, just before closing in step 0
# afterwards we are in step 1
assert(segment_steps[keysteps[0]] == 0)  # close
assert(segment_steps[keysteps[0]+1] == 1)  # open

def g2txt(step):
    return ["close","open"][gr_actions[step].astype(int)]

print("keysteps", keysteps)
print("demonstration type:", demonstration_type)
if demonstration_type == "grasp":
    assert(len(keysteps) == 1)
    grip_step = keysteps[0]  
elif demonstration_type == "grasp_insert":
    assert(len(keysteps) == 2)
    grip_step = keysteps[0]
    open_step = keysteps[1]
    print("grip_step", grip_step, g2txt(grip_step))
    print("open_step", open_step, g2txt(open_step))
else:
    raise ValueError

In [ ]:
# decide which frames to keep, after gripping mask a few steps
if keep_frames_method == "all":
    keep_array = np.ones(gr_actions.shape, dtype=bool)
elif keep_frames_method == "dense":
    keep_array = np.ones(gr_actions.shape, dtype=bool)
    keep_array[grip_step:grip_step+gripper_close_steps] = False
elif keep_frames_method == "sparse":
    keep_array = np.zeros(gr_actions.shape, dtype=bool)
    keep_array[::DEMO_STEP] = True
    keep_array[keysteps] = True
    keep_array[grip_step+1:grip_step+1+gripper_close_steps] = False  
else:
    raise ValueError

keep_array_old = keep_array.copy()
# manuall skip a few steps
if "keep_frames_skip" in locals():
    print("skipping frames:", keep_frames_skip)
    skip = np.zeros(gr_actions.shape, dtype=int)
    for (start, stop) in keep_frames_skip:
        skip[start] = 1
        skip[stop] = -1
    skip = np.cumsum(skip).astype(bool)
    keep_array[start-1] = True
    keep_array[skip] = False

if movement_threshold is not None:
    # append a entry here as diff has of n-1
    movement = np.linalg.norm(np.diff(ee_positions[:,0:3], axis=0, append=((0,0,0),)), axis=1)
    movement_mask = movement < movement_threshold    
    keep_array[movement_mask] = False

# double check that we retain all keep steps
assert(np.all(keep_array[keysteps]))

keep_fn = flow_recording_fn.replace(".npz", "_keep.npz")
np.savez(keep_fn, keep=keep_array, key=keysteps)
print("Saved to", keep_fn)

### Verify keep frames

In [ ]:
num_frames = flow_recording.shape[0]-1
x = np.linspace(0, 2 * np.pi)
fig, (ax, ax2) = plt.subplots(2, 1)
line = ax.imshow(flow_recording[0])
ax.set_axis_off()
ax2.plot(state_recording[:, -2], label="grip raw")
ax2.plot(segment_steps/10,label="steps")
ax2.plot(keep_array, label="keep")
ax2.plot((gr_actions+1)/2, label="gripper action")
ax2.set_ylabel("value")
ax2.set_xlabel("frame number")
vline = ax2.axvline(x=2, color="k")
ax2.legend()
def update(w):
    vline.set_data([w,w], [0,1])
    line.set_data(flow_recording[w])
    fig.canvas.draw_idle()
slider_w = widgets.IntSlider(min=0, max=num_frames, step=1, value=0,
                             layout=Layout(width='70%'))
interact(update, w=slider_w)

# Mask View

Mask out the foreground object so that foreground specific flow can be calculated.

In [ ]:
from skimage import measure
from scipy import ndimage

def erode_mask(mask):
    mask = ndimage.binary_closing(mask, iterations=2)
    mask = ndimage.morphology.binary_erosion(mask, iterations=4)
    return mask

def label_mask(mask, i):
    # this computes the connected components
    labels, num = measure.label(mask, background=0, return_num=True)
    label_id, label_count = np.unique(labels, return_counts=True)
    # find the biggest component here.
    #np.argsort(label_count)
    #print(label_count)
    mask = (labels == 0)
    return mask

    # create a segmentation mask
def get_mask(frame, i=None, threshold=0):
    """
    create segmentation mask for single frame
    Args:
        frame
        i index of frame
        threshold
    Returns:
        mask: binary numpy array, with False == keep
    """    
    step = segment_steps[i]
    image = frame.copy()
    color_choice = segment_colors[step]
    print("cc", color_choice)
    
    if color_choice == "bw":  # this is for the wheel task
        tmp =  np.linalg.norm(image/255, axis=2) / 3**.5
        mask = tmp > threshold
    elif color_choice == "keep_black":
        tmp =  np.linalg.norm(image/255, axis=2) 
        mask = tmp < threshold        
    else:
        color_choice = np.array(color_choice)    
        tmp = np.linalg.norm(image * color_choice, axis=2) / np.linalg.norm(image, axis=2) 
        mask = tmp > threshold
    
    mask = erode_mask(mask)
    
    if segment_height and segment_height[step]:
        depth2 = transform_depth(depth[i], np.linalg.inv(T_tcp_cam))
        mask2 = get_mask_depth(depth2, 600, 1550)
        mask[mask2] = True
    
    if segment_labels:
        mask = ndimage.morphology.binary_closing(mask, iterations=4)
        mask = label_mask(mask, i)
    
    if segment_imgheight and segment_imgheight[step]:
        value = segment_imgheight[step]
        mask[:value,:] = False
    
    if step == 0:
#         plt.imshow(mask)
        mask_lefthalf = np.zeros(mask.shape)
        mask_lefthalf[:, :330] = True
        mask = np.logical_not(np.logical_not(mask) * mask_lefthalf)
#         plt.imshow(mask)
        
    
    return mask

#
# Plot 
print("Colored stuff is keept - mask==True")
print("Segments:", keysteps)
fig, ax = plt.subplots(1, 1)
line = ax.imshow(flow_recording[25])
ax.set_axis_off()
def update(i, t):
    # detect first frame of next step
    first_frame = i==0 or i-1 in keysteps
    if first_frame:
        step = segment_steps[i]
        new_t = segment_thresholds[step]
        print("setting", new_t)
        slider_t.value = new_t*100

    image = flow_recording[i].copy()
    mask = get_mask(image, i=i, threshold=t/100)    
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()

slider_i = widgets.IntSlider(min=0, max=num_frames, step=1, value=0,
                             layout=Layout(width='70%'))
slider_t = widgets.IntSlider(min=0, max=100, step=1, value=segment_thresholds[0]*100,
                             layout=Layout(width='70%'))
interact(update, i=slider_i, t=slider_t)

print("len", len(segment_colors))

In [ ]:
masks = np.zeros(flow_recording.shape[:3], dtype=bool)
switch_frame = grip_step
print("switching at:", switch_frame)
for c,t in zip(segment_colors, segment_thresholds):
    print(c,t)
print()

for i in range(len(flow_recording)):
    try:
        step = segment_steps[i]
        threshold = segment_thresholds[step]
    except IndexError:
        break
    mask = get_mask(flow_recording[i], i, threshold)
    masks[i] = mask

print(np.mean(masks) * 100, "percent of pixels fg")
if folder_format == 'max':
    mask_fn = flow_recording_fn.replace("_img.npz", "_mask.npz")
else:
    mask_fn = flow_recording_fn.replace(".npz", "_mask.npz")
np.savez(mask_fn, mask=masks)
print("Saved to", mask_fn)

### Verify masking results

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = flow_recording[i].copy()
    mask = masks[i]
    print("mask shape", mask.shape)
    image[np.logical_not(mask)] = 255, 255, 255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0, max=num_frames, step=1, value=25,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)

# Masking based on Depth

In [ ]:
episode_data = np.load(state_recording_fn)
#keys = list(episode_data.keys())
camera_calibration = dict(width=640,height=480,
                     fx = 617.8902587890625, fy=617.8903198242188, 
                     ppx=315.20367431640625, ppy=245.70614624023438 )


T_tcp_cam = np.array([
    [0.99987185, -0.00306941, -0.01571176, 0.00169436],
    [-0.00515523, 0.86743151, -0.49752989, 0.11860651],
    [0.015156,    0.49754713,  0.86730453, -0.18967231],
    [0., 0., 0., 1.]])

depth = episode_data["depth_imgs"]
depth_scale = 8000

i = 200
print("loaded.")

In [ ]:
def transform_depth(depth_image, transformation):
    """
    Transform a depth image into a point cloud with one point for each
    pixel in the image, using the camera transform for a camera
    centred at cx, cy with field of view fx, fy.

    depth is a 2-D ndarray with shape (rows, cols) containing
    depths from 1 to 254 inclusive. The result is a 3-D array with
    shape (rows, cols, 3). Pixels with invalid depth in the input have
    NaN for the z-coordinate in the result.
    """
    assert(camera_calibration)
    assert(camera_calibration["width"] == depth_image.shape[1])
    assert(camera_calibration["height"] == depth_image.shape[0])

    C_X = camera_calibration["ppx"]
    C_Y = camera_calibration["ppy"]
    F_X = camera_calibration["fx"]
    F_Y = camera_calibration["fy"]
    
    rows, cols = depth_image.shape
    c, r = np.meshgrid(np.arange(cols), np.arange(rows), sparse=True)
    
    z = depth_image
    x = z * (c - C_X) / F_X
    y = z * (r - C_Y) / F_Y
    o = np.ones(z.shape)
    
    tmp = np.stack((x, y, z, o), axis=2)
    tmp2 = tmp @ transformation
    tmp3 = tmp2[:, :, :3] / tmp2[:, :, 3, np.newaxis]
    return tmp3[:,:,2]


depth_flat = transform_depth(depth[i], np.linalg.inv(T_tcp_cam))
fig, (ax,ax2) = plt.subplots(1,2)
line = ax.imshow(depth_flat)
ax2.plot(np.sort(depth[i].flatten()))
ax2.plot(np.sort(depth_flat.flatten()))
plt.show()

In [ ]:
def get_mask_depth(frame, l, h):
    mask = np.logical_or(frame < l/depth_scale, frame > h/depth_scale)
    mask = np.logical_not(mask)
    return mask

def erode_mask(mask):
    return mask
    mask = ndimage.binary_closing(mask, iterations=5)
    mask = ndimage.morphology.binary_erosion(mask, iterations=10)
    return mask

x = np.linspace(0, 2 * np.pi)
fig, ax = plt.subplots(1)
line = ax.imshow(flow_recording[0])

def update(w,l,h):
    depth2 = transform_depth(depth[w], np.linalg.inv(T_tcp_cam))
    mask = get_mask_depth(depth2, l, h)
    mask = erode_mask(mask)
    mask = np.logical_not(mask)
    display_image = flow_recording[w].copy()
    display_image[mask] = 0
    line.set_data(display_image)
    fig.canvas.draw_idle()
    
depth_min, depth_max = int(depth.min()*depth_scale), int(depth.max()*depth_scale)
slider_w = widgets.IntSlider(min=0,max=num_frames,step=1,value=205,
                             layout=Layout(width='70%'))
slider_l = widgets.IntSlider(min=depth_min,max=depth_max,step=1,value=1560,
                             layout=Layout(width='70%'))
slider_h = widgets.IntSlider(min=depth_min,max=depth_max,step=1,value=1650,
                             layout=Layout(width='70%'))

interact(update,w=slider_w,l=slider_l,h=slider_h)

In [ ]:
# next steps: anneal the edge, and run connected component algorithm.
from scipy import ndimage

w = slider_w.value
l = slider_l.value
h = slider_h.value
print("w={w}, l={l}, h={h}".format(w=w,l=l,h=h))

depth2 = transform_depth(depth[w], np.linalg.inv(T_tcp_cam))
mask_s = get_mask_depth(depth2, l, h)
mask_s = erode_mask(mask_s.copy())
mask_s = np.logical_not(mask_s)
display_image = flow_recording[w].copy()
display_image[mask_s] = 0

fig, ax = plt.subplots(1)
line = ax.imshow(display_image)
plt.show()



In [ ]:
threshold_low = slider_l.value
threshold_high = slider_h.value

masks = np.zeros(flow_recording.shape[:3],dtype=bool)

for i in range(len(flow_recording)):
    mask = get_mask_depth(depth[i], threshold_low, threshold_high)
    mask = erode_mask(mask)
    masks[i] = mask

print(np.mean(masks) * 100, "percent of pixels fg")
mask_fn = flow_recording_fn.replace(".npz","_mask.npz")
np.savez(mask_fn, mask=masks)
print("Saved to",mask_fn)

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = flow_recording[i].copy()
    mask = masks[i]
    image[mask] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=200,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)